In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#trying to fix certificate issues

#import certifi
#import requests
 
#WEB_SITE='http://api.semanticscholar.org/' # Website to test
#NM_ROOT_CA='NMProdCACertBundle.pem' # Pem file from above link
#try:
    #print('Checking connection...')
    #test = requests.get(WEB_SITE)
    #print('Connection OK.')
#except requests.exceptions.SSLError as err:
   # print('SSL Error. Adding custom certs to Certifi store...')
    #cafile = certifi.where()
    #with open(NM_ROOT_CA, 'rb') as infile:
        #customca = infile.read()
    #with open(cafile, 'ab') as outfile:
        #outfile.write(customca)
    #print('That might have worked.')

In [ ]:
import requests
import pandas as pd

url = 'https://api.nsf.gov/services/v1/awards.json'
params = {
    'printFields': 'agency,awardeestatecode,coPDPI,startDate,expDate,estimatedTotalAmt,abstractText,awardeeStateCode',
    'keyword': 'artificial intelligence', 'offset': 0
}

df_list = [] # Initialize an empty list to store DataFrames for each page of results

while True:
    response = requests.get(url, params=params, verify='NMProdCACertBundle.pem')

    if response.status_code == 200:
        results = response.json()['response']['award']
        df = pd.DataFrame.from_records(results)
        df_list.append(df)

        if len(results) < 25:
            # If the page contains less than 25 records, we've reached the end of the results
            break
        else:
            # Otherwise, increase the offset to retrieve the next page of results
            params['offset'] += 25
    else:
        print('Error:', response.status_code)
        break

df = pd.concat(df_list, ignore_index=True) # Combine all DataFrames into a single DataFrame
print(df)


In [ ]:
df.to_csv('NSF_awards_AI.csv')

In [ ]:
df.info()

In [ ]:
import datetime

In [ ]:
#filter down to last 5 years of data
from datetime import datetime, timedelta
# Convert startDate and expDate columns to datetime type
df['startDate'] = pd.to_datetime(df['startDate'])
df['expDate'] = pd.to_datetime(df['expDate'])
# Filter DataFrame to include rows with a startDate from the last 5 years
five_years_ago = datetime.now().date() - timedelta(days=365*5)
NSF_last5 = df[df['startDate'] >= pd.to_datetime(five_years_ago)]
NSF_last5.head()

In [ ]:
NSF_last5.info()

In [ ]:
NSF_last5.to_csv('NSF_awards_AI_last5.csv')

In [ ]:
NSF_WI_last5 = NSF_last5[NSF_last5['awardeeStateCode']=='WI']
NSF_WI_last5.head()

In [ ]:
NSF_WI_last5.to_csv('WI_NSF_Awards_AI_last5.csv')

In [ ]:
#reloading the saved datasets for some analysis and viz
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
NSF_last5 = pd.read_csv('NSF_awards_AI_last5.csv')

In [ ]:
NSF_last5.head()

In [ ]:
NSF_last5.info()

In [ ]:
NSF_last5.columns

In [ ]:
NSF_last5.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
pd.options.display.float_format = '{:.0f}'.format
NSF_last5.describe()

In [ ]:
# Group the DataFrame by awardeestatecode and sum the estimatedTotalAmt column, and sort in descending order
grouped = NSF_last5.groupby('awardeeStateCode')['estimatedTotalAmt'].sum().sort_values(ascending=False)

# Define a formatter function to display values in millions


# Create a bar plot of the grouped data
plt.figure(figsize=(20, 5)) # Set the figure size to be twice as long
plt.bar(grouped.index, grouped.values)

# Set the x-axis label rotation to 90 degrees
plt.xticks(rotation=90)

# Set the plot title and axis labels
plt.title('Estimated Total Awards by State')
plt.xlabel('Awardee State Code')
plt.ylabel('Estimated Total Awards (in hundreds of millions)')

# Show the plot
plt.show()

In [ ]:
# Filter the DataFrame to include only rows where awardeeStateCode = 'CA'
CA_df = NSF_last5[NSF_last5['awardeeStateCode'] == 'CA']

# Compute the sum of estimatedTotalAmt for the filtered DataFrame
total_amt_CA = CA_df['estimatedTotalAmt'].sum()

print(f'The total estimated award amount for awardeeStateCode CA is ${total_amt_CA:.2f}.')

In [ ]:
pip install bertopic

In [ ]:
from bertopic import BERTopic